자연어 전처리

In [3]:
!pip uninstall torchtext
!pip install torchtext==0.4.0

Found existing installation: torchtext 0.4.0
Uninstalling torchtext-0.4.0:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/torchtext-0.4.0.dist-info/*
    /usr/local/lib/python3.8/dist-packages/torchtext/*
Proceed (Y/n)? y
  Successfully uninstalled torchtext-0.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached torchtext-0.4.0-py3-none-any.whl (53 kB)


In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchtext import data,datasets


# 하이퍼파라미터
BATCH_SIZE = 64
lr = 0.001
EPOCHS = 10
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("다음 기기로 학습합니다:", DEVICE)



다음 기기로 학습합니다: cuda


In [2]:
#전처리 정의
TEXT=data.Field(sequential=True,batch_first=True,lower=True) # sequential : 순차적이면 True , batch_first : True면 신경망에 입력되는 텐서의 첫 번째 차원값이 batch_size
LABEL=data.Field(sequential=False,batch_first=True) 

#학습셋과 테스트셋
trainset,testset=datasets.IMDB.splits(TEXT,LABEL) # TabularDataset : text와 label을 속성으로 가짐

#워드 임베딩에 필요한 단어 사전
TEXT.build_vocab(trainset,min_freq=5) # 최소 5번 이상 등장한 단어만 사전에 담겠다는 것  # buld_vocab : 정수 인코딩 작업 전에 단어들의 집합 생성
LABEL.build_vocab(trainset)

In [3]:
# valset (훈련셋의 20%를 검증셋으로)
trainset,valset=trainset.split(split_ratio=0.8)
train_iter,val_iter,test_iter=data.BucketIterator.splits((trainset,valset,testset),batch_size=BATCH_SIZE,shuffle=True,repeat=False)


In [4]:
vocab_size=len(TEXT.vocab) # 사전 속 단어들의 개수
n_classes=2


In [5]:
print('[학습셋]: %d [검증셋] :%d [테스트셋]:%d [단어수]:%d [클래스]:%d'%(len(trainset),len(valset),len(testset),vocab_size,n_classes))

[학습셋]: 20000 [검증셋] :5000 [테스트셋]:25000 [단어수]:46159 [클래스]:2


In [6]:
#trainset 구성(TabularDataset)
print(trainset[0].text,trainset[0].label)

['first', 'off', 'there', 'is', 'nothing', 'wrong', 'with', 'studying', 'daniel,', 'revelation,', 'matthew', '24,', 'isaiah', 'and', 'other', 'prophetic', 'scriptures.', 'there', 'is', 'also', 'nothing', 'wrong', 'with', 'making', 'a', 'film', 'such', 'as', 'this', 'to', 'attempt', 'to', 'present', 'the', 'gospel', 'message.', 'so', 'my', 'qualms', 'with', 'this', 'movie', 'are', 'not', 'in', 'either', 'its', 'sincerity', 'or', 'aspirations.', 'as', 'a', 'christian,', 'though', 'an', 'amillenialist,', 'i', 'believe', 'there', 'will', 'be', 'a', 'great', 'tribulation', 'and', 'i', 'believe', 'christ', 'will', 'return', 'as', 'he', 'said', 'as', 'much.', 'so', 'even', 'though', 'i', 'have', 'disagreement', 'with', 'this', 'film', 'about', 'the', 'rapture', 'that', 'is', 'not', 'why', 'i', 'rate', 'this', 'movie', 'so', 'low.<br', '/><br', '/>no,', 'what', 'makes', 'me', 'rate', 'this', 'movie', 'so', 'low', 'is', 'not', 'its', 'sincerity', 'or', 'its', 'message,', 'but', 'rather', 'its',

RNN 모델 구현

In [11]:
class BasicGRU(nn.Module):
  def __init__(self,n_layers,hidden_dim,n_vocab,embed_dim,n_classes,dropout_p=0.2):
    super(BasicGRU,self).__init__()
    print('Buliding Basic GRU model...')

    self.n_layers=n_layers
    self.embed=nn.Embedding(n_vocab,embed_dim) # 단어 집합 개수만큼 임베딩 벡터를 가진 임베딩 테이블 (자연어 -> 임베딩 벡터로 맵핑)

    self.hidden_dim=hidden_dim # 은닉벡터의 차원값
    self.dropout=nn.Dropout(dropout_p)

    self.gru=nn.GRU(embed_dim,self.hidden_dim,num_layers=self.n_layers,batch_first=True) # RNN : 입력이 길어지면 학습도중 기울기가 너무 커지거나 작아지는 현상이 발생, gru : rnn의 단점을 보완하여 기울기를 적정하게 유지하고 문장 앞부분의 정보가 뒷부분에도 전달될 수 있게 함
    self.out=nn.Linear(self.hidden_dim,n_classes)

  def forward(self,x):
        x=self.embed(x)
        h_0=self._init_state(batch_size=x.size(0)) # 첫 번째 은닉벡터, RNN 신경망은 입력과 첫번째 은닉벡터를 입력해줘야함 
        x, _ = self.gru(x, h_0)  # [i, b, h]
        h_t = x[:,-1,:] # 모든 리뷰들을 압축한 은닉벡터 
        self.dropout(h_t)
        logit = self.out(h_t)  # [b, h] -> [b, o] #
        return logit # 결과 출력 

  def _init_state(self,batch_size=1):
      weight=next(self.parameters()).data
      return weight.new(self.n_layers,batch_size,self.hidden_dim).zero_()
    




In [12]:
def train(model,optimizer,train_iter):
    model.train()
    for b,batch in enumerate(train_iter): # 반복마다 배치데이터 반환
        x,y=batch.text.to(DEVICE),batch.label.to(DEVICE) # text : 영화평 데이터, label : 레이블 

        y.data.sub_(1) # label을 0,1로 바꿈

        optimizer.zero_grad() # 기울기 0으로 초기화
        logit=model(x) # 예측값

        loss=F.cross_entropy(logit,y) # 오차
        loss.backward()
        optimizer.step() # 최적화 


def evaluate(model,val_iter):
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy


In [13]:
#모델 객체 정의
model=BasicGRU(1,256,vocab_size,128,n_classes,0.5).to(DEVICE)  # 은닉벡터의 차원값 256, 임베딩된토큰의 차원값 128
optimizer=torch.optim.Adam(model.parameters(),lr=lr)

Buliding Basic GRU model...


In [14]:
# 최종 모델 : 검증오차가 가장 작은 모델
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[이폭: %d] 검증 오차:%5.2f | 검증 정확도:%5.2f" % (e, val_loss, val_accuracy))
    
    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification.pt')
        best_val_loss = val_loss

[이폭: 1] 검증 오차: 0.70 | 검증 정확도:50.34
[이폭: 2] 검증 오차: 0.68 | 검증 정확도:58.40
[이폭: 3] 검증 오차: 0.44 | 검증 정확도:81.58
[이폭: 4] 검증 오차: 0.33 | 검증 정확도:87.00
[이폭: 5] 검증 오차: 0.32 | 검증 정확도:87.22
[이폭: 6] 검증 오차: 0.34 | 검증 정확도:86.84
[이폭: 7] 검증 오차: 0.35 | 검증 정확도:87.24
[이폭: 8] 검증 오차: 0.39 | 검증 정확도:86.62
[이폭: 9] 검증 오차: 0.41 | 검증 정확도:86.38
[이폭: 10] 검증 오차: 0.42 | 검증 정확도:86.70


In [15]:
#테스트 셋으로 모델 성능 평가

model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.33 | 테스트 정확도: 86.62
